# Install packages

In [1]:
%%capture
%%bash
python -m pip install -U pip numpy vaex-core vaex-hdf5
python -m pip install -U ipython ipykernel

# Please restart the karnel after installation 

In [13]:
import gc
import numpy as np
import pandas as pd
import warnings
import time
import gc
import os

instance_type = 'c5d4xlarge' # change this
results_bucket = f"s3://xdss-benchmarks/benchmarks" # change this

name = 'vaex'
data_path = 'datasets/taxi_1B.hdf5'
output_file = f'{name}_{instance_type}.csv'
results_path = f"results/{output_file}"
results_bucket = f"{results_bucket}/{output_file}" 
benchmarks = {
    'run':[],
    'duration': [],
    'task': []   
}

long_min = -74.05
long_max = -73.75
lat_min = 40.58
lat_max = 40.90


def get_results(benchmarks):
    return pd.DataFrame.from_dict(benchmarks, orient='index').T

def persist():
    gc.collect()
    get_results(benchmarks).to_csv(results_path)
    os.system(f"aws s3 cp {results_path} {results_bucket}")
    
def benchmark(f, df, name, **kwargs):    
    for i in range(2):
        start_time = time.time()
        ret = f(df, **kwargs)
        benchmarks['duration'].append(time.time()-start_time)
        benchmarks['task'].append(name)
        benchmarks['run'].append(i+1)
    persist()
    print(f"{name} took: {benchmarks['duration'][-1]}")
    return benchmarks['duration'][-1]


!mkdir -p results
!mkdir -p datasets
print(f"We test every benchmark twice and save both results")

We test every benchmark twice and save both results


# Benchmark

In [14]:
import vaex
import numpy as np

data = vaex.open(data_path)
print(f"size: {len(data)} with {len(data.columns)} columns")

size: 1173057927 with 18 columns


In [15]:
def open_file(df=None):
    return vaex.open(data_path)

benchmark(open_file, df=data, name='read_file')

read_file took: 0.005378246307373047


0.005378246307373047

In [16]:
def count(df=None):
    return len(df)

benchmark(count, df=data, name='count')

count took: 2.384185791015625e-06


2.384185791015625e-06

In [17]:
def mean(df):
    return df.fare_amount.mean()

benchmark(mean, df=data, name='mean')

mean took: 0.5676648616790771


0.5676648616790771

In [18]:
def standard_deviation(df):
    return df.fare_amount.std()

benchmark(standard_deviation, df=data, name='standard deviation')

standard deviation took: 1.7552824020385742


1.7552824020385742

In [19]:
def mean_of_sum(df):
    return (df.fare_amount + df.trip_distance).mean()

benchmark(mean_of_sum, df=data, name='sum columns mean')

sum columns mean took: 0.7302772998809814


0.7302772998809814

In [20]:
# lazy evaulation - instant
def sum_columns(df):
    return (df.fare_amount + df.trip_distance)

benchmark(sum_columns, df=data, name='sum columns')

sum columns took: 7.867813110351562e-06


7.867813110351562e-06

In [21]:
def mean_of_product(df):
    return (df.fare_amount * df.trip_distance).mean()

benchmark(mean_of_product, df=data, name='product columns mean')

product columns mean took: 0.7427091598510742


0.7427091598510742

In [22]:
# lazy evaulation - instant
def product(df):
    return df.fare_amount * df.trip_distance

benchmark(product, df=data, name='product columns')

product columns took: 7.3909759521484375e-06


7.3909759521484375e-06

In [23]:
def lazy_mean(df):
    df['lazy'] = df.fare_amount * df.trip_distance
    return df['lazy'].mean()
    
benchmark(lazy_mean, df=data, name='lazy evaluation')  

lazy evaluation took: 0.7424013614654541


0.7424013614654541

In [24]:
def value_counts(df):
    return df.passenger_count.value_counts()

benchmark(value_counts, df=data, name='value counts')

value counts took: 0.33688783645629883


0.33688783645629883

In [25]:
def mean_of_complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    df['complicated'] = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return df['complicated'].mean()

benchmark(mean_of_complicated_arithmetic_operation, df=data, name='arithmetic operation mean')


arithmetic operation mean took: 10.441593408584595


10.441593408584595

In [26]:
# lazy evaulation - instant
def complicated_arithmetic_operation(df):
    theta_1 = df.pickup_longitude
    phi_1 = df.pickup_latitude
    theta_2 = df.dropoff_longitude
    phi_2 = df.dropoff_latitude
    temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
           + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
    df['complicated'] = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
    return df['complicated']

benchmark(complicated_arithmetic_operation, df=data, name='arithmetic operation')

arithmetic operation took: 0.004083156585693359


0.004083156585693359

In [27]:
def groupby_statistics(df):
    return df.groupby(by='passenger_count').agg({'fare_amount': ['mean', 'std'], 
                                               'tip_amount': ['mean', 'std']
                                              })

benchmark(groupby_statistics, df=data, name='groupby statistics')

groupby statistics took: 6.03224515914917


6.03224515914917

In [28]:
other = groupby_statistics(data)

In [29]:
def join_count(df, other):
    return len(df.join(other=other, on = 'passenger_count', rsuffix = '_right'))

benchmark(join_count, data, name='join count', other=other)

join count took: 0.8022336959838867


0.8022336959838867

In [30]:
def join_data(df, other):
    return df.join(other=other, on = 'passenger_count', rsuffix = '_right')

benchmark(join_data, data, name='join', other=other)

join took: 0.4697263240814209


0.4697263240814209

## Filtered data

Dask is not build to run on filter data like you would normally, so we will apply the same strategy

In [31]:
print(f"Prepare filtered data and deleted {gc.collect()} MB")

Prepare filtered data and deleted 1054 MB


In [32]:
def filter_data(df):
    expr_filter = (df.pickup_longitude > long_min)  & (df.pickup_longitude < long_max) & \
                  (df.pickup_latitude > lat_min)    & (df.pickup_latitude < lat_max) & \
                  (df.dropoff_longitude > long_min) & (df.dropoff_longitude < long_max) & \
                  (df.dropoff_latitude > lat_min)   & (df.dropoff_latitude < lat_max)
    return df[expr_filter]

benchmark(filter_data, data, name='filter data')

filter data took: 1.2630045413970947


1.2630045413970947

In [33]:
filtered = filter_data(data)

del data
print(f"cleaned {gc.collect()} mb")

cleaned 2887 mb


In [34]:
benchmark(mean, filtered, name='filtered mean')
benchmark(standard_deviation, filtered, name='filtered standard deviation')
benchmark(mean_of_sum, filtered, name ='filtered sum columns mean')
benchmark(sum_columns, df=filtered, name='filtered sum columns')
benchmark(mean_of_product, filtered, name ='filtered product columns mean')
benchmark(product, df=filtered, name='filtered product columns')
benchmark(mean_of_complicated_arithmetic_operation, filtered, name='filtered arithmetic operation mean')
benchmark(complicated_arithmetic_operation, filtered, name='filtered arithmetic operation')
benchmark(value_counts, filtered, name ='filtered value counts')
benchmark(groupby_statistics, filtered, name='filtered groupby statistics')
other = groupby_statistics(filtered)
benchmark(join_count, filtered, name='filtered join count', other=other)
benchmark(join_data, filtered, name='filtered join', other=other)
print('vaex')
get_results(benchmarks)

<string>:1: RuntimeWarning: invalid value encountered in greater
<string>:1: RuntimeWarning: invalid value encountered in less


filtered mean took: 0.9566733837127686
filtered standard deviation took: 2.0329854488372803
filtered sum columns mean took: 2.7238519191741943
filtered sum columns took: 1.0251998901367188e-05
filtered product columns mean took: 2.7048892974853516
filtered product columns took: 9.775161743164062e-06
filtered arithmetic operation mean took: 13.190029621124268
filtered arithmetic operation took: 0.004437446594238281
filtered value counts took: 0.625408411026001
filtered groupby statistics took: 10.025638580322266
filtered join count took: 0.07559323310852051
filtered join took: 0.07555079460144043
vaex


,run,duration,task
0,1,0.00634384,read_file
1,2,0.00537825,read_file
2,1,4.02927e-05,count
3,2,2.38419e-06,count
4,1,0.760968,mean
5,2,0.567665,mean
6,1,1.75754,standard deviation
7,2,1.75528,standard deviation
8,1,0.829949,sum columns mean
9,2,0.730277,sum columns mean
